In [1]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *


In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [3]:
df>>count(_.mpo)>>arrange(-_.n)

,mpo,n
18,SCAG,4988
12,MTC,3236
13,NON-MPO,2592
14,SACOG,1865
2,CFCG,1404
8,KCOG,998
26,STANCOG,759
0,AMBAG,745
21,SJCG,515
29,TCAG,489


In [4]:
df >> count(_.mpo)>>arrange(-_.n)>> filter(_.n==1)

,mpo,n
6,FSTIP,1
11,MNOLTC,1
15,SANDA,1
20,SHASTA,1


In [5]:
df>>filter(_.mpo =='SANDA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA,2013-01-01


* maybe meant "SANDAG" but Big Bear Lake is in "SCAG" jurisdiction
* looked to excel download file and this there is anothe Big Bear Lake and SANDAG instance

In [6]:
df>>filter(_.agency.str.contains('Big Bear')) >> filter(_.mpo.str.contains("SANDA"))

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12567,HSIPLN,5426(011),Big Bear Lake,2014-02-20,27000.0,0.0,30000.0,Authorized,NaN,8,SCAG015,"Big Bear Boulevard (sr 18) From 1,000 Ft. West...",Widen Shoulder For Two Way Left Turn Lane,1,SANDAG,2014-01-01
12767,HSIPLN,5426(008),Big Bear Lake,2013-12-09,169650.0,0.0,188924.0,Authorized,NaN,8,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,SANDA,2013-01-01


In [7]:
df >>filter(_.mpo=='SHASTA')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10037,HSIP,5073(075),Orange,2014-12-30,196199.0,0.0,218000.0,Authorized,NaN,12,SCAG015,Intersection Of Tustin Street And Mayfair Avenue,Left-turn Signal Phasing,2,SHASTA,2014-01-01


In [8]:
df>> filter(_.agency.str.contains('Shasta')) >> count(_.mpo)

,mpo,n
0,MTC,1
1,SACOG,3
2,STPA,129


In [9]:
df>>filter(_.agency.str.contains('Shasta')) >> filter(_.mpo=="MTC")

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
19168,ACSTP,15A6(002),Shasta County,2021-05-18,0.0,19251.73,21746.0,Authorized,5921,2,NaN,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,MTC,2021-01-01


* in the project location column, the intersection is Tustin and Mayfair which is in Orange Couny, or SCAG. Agency is also Orange, yet the mpo is Shasta

In [10]:
df>> filter(_.mpo=='MNOLTC')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10820,RPSTP,5205(019),Rialto,2014-09-10,2000000.0,-2000000.0,0.0,Authorized,NaN,8,20040830,Former Pacific Electric Railroad Right-of-way ...,Extend Class 1 Bicycle Trail,2,MNOLTC,2014-01-01


In [11]:
df>> filter(_.agency=='Rialto') >> count(_.mpo)

,mpo,n
0,KCOG,1
1,MNOLTC,1
2,SANDAG,1
3,SCAG,15


In [12]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='KCOG')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
10667,SRTS,5205(014),Rialto,2014-10-08,0.0,0.0,85287.07,Authorized,NaN,8,SCAG016,"Eucalyptus Avenue, King Street, 2nd Street, En...","Curb, Ramps, And In-pavement Warning Signals",3,KCOG,2014-01-01


* interesting. wondering if there are other instances of this

In [13]:
df>>filter(_.mpo=='FSTIP')

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
12011,CML,5160(023),Los Banos,2014-04-16,-16940.31,0.0,-29901.84,Authorized,NaN,10,CMAQ11-05,In City Of Los Banos,Construct/install Pedestrian/bike Improvement,4,FSTIP,2014-01-01


* FSTIP is the Federal Statewide Transportation Improvement Program. 